In [1]:
import requests
import os
import json
from dotenv import load_dotenv
load_dotenv()

# --- 1. FRED API 설정 정보 ---

# 데이터 시리즈 ID (미국 금리 핵심 지표)
SERIES_IDS = {
    "기준금리 (FEDFUNDS)": "FEDFUNDS",  # 실효 연방기금 금리 (단기 정책 금리)
    "10년물 국채 수익률 (DGS10)": "DGS10",  # 장기 시장 금리 (시장 벤치마크)
}

# 요청 파라미터 설정
# - observation_start: 데이터 시작 날짜 (YYYY-MM-DD)
# - file_type: 응답 데이터 형식 (json이 분석에 용이)
# - frequency: 데이터 주기 (d: 일별, m: 월별, q: 분기별, a: 연간)
REQUEST_PARAMS = {
    # 💡 2015년 10월 1일부터 요청 (약 10년치)
    "observation_start": "2015-10-01", 
    "file_type": "json",
    "api_key" : os.getenv("FRED_API_KEY"), # 환경 변수에서 FRED_API_KEY를 가져와 설정
    "frequency": "m"  # 월별 데이터 요청
}

def get_fred_data(series_id, series_name):
    """
    FRED API에서 특정 시리즈 ID의 데이터를 가져오는 함수
    """
    # FRED API의 기본 URL
    base_url = "https://api.stlouisfed.org/fred/series/observations"
    
    # 시리즈 ID 추가
    params = REQUEST_PARAMS.copy()
    params["series_id"] = series_id

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status() # HTTP 오류가 발생하면 예외 발생
        
        data = response.json()
        
        print(f"\n--- {series_name} 데이터 ({series_id}) ---")
        
        # 데이터가 있는지 확인
        observations = data.get("observations")
        if not observations:
            print("데이터를 찾을 수 없거나 API 키가 잘못되었습니다.")
            return

        # 💡 최근 12개월 (1년치) 데이터만 출력합니다. (API는 10년치를 가져옴)
        print("최근 12개월 데이터 출력:")
        for obs in observations[-12:]:
            print(f"날짜: {obs['date']}, 금리: {obs['value']}%")

    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
    except json.JSONDecodeError:
        print("JSON 디코딩 오류: 응답 형식이 잘못되었습니다.")

# --- 메인 실행 ---
if __name__ == "__main__":
    for name, series_id in SERIES_IDS.items():
        get_fred_data(series_id, name)


--- 기준금리 (FEDFUNDS) 데이터 (FEDFUNDS) ---
최근 12개월 데이터 출력:
날짜: 2024-10-01, 금리: 4.83%
날짜: 2024-11-01, 금리: 4.64%
날짜: 2024-12-01, 금리: 4.48%
날짜: 2025-01-01, 금리: 4.33%
날짜: 2025-02-01, 금리: 4.33%
날짜: 2025-03-01, 금리: 4.33%
날짜: 2025-04-01, 금리: 4.33%
날짜: 2025-05-01, 금리: 4.33%
날짜: 2025-06-01, 금리: 4.33%
날짜: 2025-07-01, 금리: 4.33%
날짜: 2025-08-01, 금리: 4.33%
날짜: 2025-09-01, 금리: 4.22%

--- 10년물 국채 수익률 (DGS10) 데이터 (DGS10) ---
최근 12개월 데이터 출력:
날짜: 2024-11-01, 금리: 4.36%
날짜: 2024-12-01, 금리: 4.39%
날짜: 2025-01-01, 금리: 4.63%
날짜: 2025-02-01, 금리: 4.45%
날짜: 2025-03-01, 금리: 4.28%
날짜: 2025-04-01, 금리: 4.28%
날짜: 2025-05-01, 금리: 4.42%
날짜: 2025-06-01, 금리: 4.38%
날짜: 2025-07-01, 금리: 4.39%
날짜: 2025-08-01, 금리: 4.26%
날짜: 2025-09-01, 금리: 4.12%
날짜: 2025-10-01, 금리: .%
